<a href="https://colab.research.google.com/github/akash-kaul/DemoLoaderTigerGraphGraphathon/blob/master/DemoLoaderTG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import packages
from ipywidgets import interact, interactive, fixed, interact_manual, widgets, Layout
from IPython.display import display, HTML
from IPython.utils import io
from tqdm import tqdm
from ipywidgets.widgets import VBox, HBox, Box, Label
import time
from glob import glob
import requests
from functools import wraps
import re

# Download CSS Sheet for Button Icons
display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))

# Create output widget for temporary displays (i.e. pop-up messages)
out = widgets.Output()

# Establish format for all boxes
form_item_layout = Layout(
    display='inline-flex',
    flex_flow='row',
    justify_content='space-between'
)

# Create a text field for hostname
hostname = widgets.Text(
    value='',
    placeholder='TigerGraph Hostname (ex: https://tg.i.tgcloud.io)',
    disabled=False
)

# Create a text field for username
username = widgets.Text(
    value='tigergraph',
    placeholder='TigerGraph Username',
    disabled=False
)

# Create a text field for password
password = widgets.Text(
    value='',
    placeholder='TigerGraph Password',
    disabled=False,
)

# Create a checkbox field for whether a cert is needed or not
cert = widgets.Checkbox(
    value=False,
    disabled=False,
    indent=False
)

# Create initial connect button to connect to server
connect = widgets.Button(
    description='Connect To TigerGraph',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to connect to graph',
    icon='plug', # (FontAwesome names without the `fa-` prefix)
    layout=Layout(width='100%')
)

# Create label for error message if can't connect to graph
error = widgets.Label(
    value='Something went wrong!! Make sure your server is live and your information is entered correctly',
    disabled=False,
    layout=Layout(width='50%')

)

# Create button to cancel creating graph due to another graph already existing (click to cancel load, keep old graph)
graph_exists_cancel = widgets.Button(
    description = 'Cancel',
    disabled=False,
    button_style='danger',
    icon='ban',
    layout=Layout(width='50%')
)

# Create button to continue creating graph even though another graph already exists (click to replace graph)
graph_exists_continue = widgets.Button(
    description = 'Continue',
    disabled=False,
    button_style='success',
    icon='arrow-circle-right',
    layout=Layout(width='50%')
)

# Create label for error message if can't create the new graph
error_load = widgets.Label(
    value='There was an error loading the data. Continuing with remaining scripts.',
    disabled=False,
    layout=Layout(width='50%')
)

# Create initial layout for input form
form_items = [
    Box([Label(value='Hostname'), hostname], layout=form_item_layout),
    Box([Label(value='Username'), username], layout=form_item_layout),
    Box([Label(value='Password'), password], layout=form_item_layout),
    Box([Label(value='Cert Needed'), cert], layout=form_item_layout),
    Box([connect], layout=form_item_layout)
]

# Method run if the first connect button is clicked
def connectToGraph(b):
    with tqdm(total=100, leave=False) as pbar:      # Create loading bar
        out.clear_output()
        pbar.bar_format="{percentage:3.0f}%|{bar:10}|{desc}"    # Set bar format

        # Change connect button from 'connect' to 'loading', chnage color from blue to orange
        connect.icon='spinner'
        connect.description='Loading'
        connect.button_style='warning'

        #--pyTigerGraph_BEGIN--#
        pbar.set_description_str('Installing pyTigerGraph')
        with io.capture_output() as captured:       # Code used to supress output to console (so doesn't show packages being installed)
            !pip install pytigergraph
            import pyTigerGraph as tg
        pbar.update(20)
        #--pyTigerGraph_END--#

        #--Establish_Connection_BEGIN--#
        pbar.set_description_str('Connecting to Server')

        # Try connecting to graph, if fails stop and print error message
        try:
            with io.capture_output() as captured:
                graph = tg.TigerGraphConnection(host=hostname.value, username=username.value, password=password.value, useCert=cert.value)
        except Exception as e:
            pbar.close()
            out.clear_output()
            with out:
                display(error)
                connect.icon='plug'
                connect.description='Connect to TigerGraph'
                connect.button_style='info'
            return
        pbar.update(20)
        #--Establish_Connection_END--#      

        #--Git_Clone_BEGIN--#
        pbar.set_description_str('Grabbing Demos from Github')
        from glob import glob
        import os
        if not os.path.exists('/content/ecosys'):
            with io.capture_output() as captured:
                !git clone https://github.com/tigergraph/ecosys.git     # Clone Demos Github Repo, store names/paths of demos
        os.chdir("/content/ecosys/demos/guru_scripts")
        names = {}
        names['Fraud Detection Demo'] = os.path.join(os.getcwd(), 'fraud_detection_demo')
        # names['Pagerank Demo'] = os.path.join(os.getcwd(), 'pagerank_demo')           -- currently doesn't work due to the format of the scripts
        pbar.update(40)
        #--Git_Clone_END--#

        #--Create_Dropdown_BEGIN--#
        pbar.set_description_str(' Collecting Scripts')

        # Create dropdown of demo options to load
        dropdown = widgets.Dropdown(
                options=names.keys(),
                disabled=False,
                style = {'description_width': 'initial'},
                )
        time.sleep(2)
        pbar.update(20)
        #--Create_Dropdown_END--#

        #--Wrap_Up_BEGIN--#     Close loading bar, disable all text boxes in the form, update button to 'success' with green color
        pbar.set_description_str(' Connected')
        connect.icon='check-circle'
        connect.description='Success'
        connect.button_style='success'
        time.sleep(1)
        pbar.close()
        username.disabled=True
        password.disabled=True
        hostname.disabled=True
        cert.disabled=True
        connect.disabled=True
        #--Wrap_Up_END--#

    #--Display_Demo_Options_And_Load_BEGIN--#

    # Grab README in Github repo for initial demo selected in dropdown (if no README exits, display appropriate message)
    x = ''
    path = names[dropdown.value]
    if os.path.exists(os.path.join(path, 'README.md')):
        full_path = os.path.join(path, 'README.md')
        with open(full_path, 'r') as file:
            x = file.read()
    else:
        x = 'No README available'
    
    # Create text field with README info to be displayed below dropdown
    text = widgets.Textarea(
        value=x,
        disabled=False,
        layout=Layout(height='300px')
    )

    # Create install button that will start installing demo when clicked
    confirm = widgets.Button(
            description='Install Demo',
            icon='cloud-upload',
            disabled=False,
            button_style='info',
            layout=Layout(width='100%'),
    )

    # Create layout for display items (matching initial form) and show in console
    form_items2 = [
        Box([Label(value='Demos'), dropdown], layout=form_item_layout),
        Box([Label(value='Info'), text], layout=form_item_layout),
        Box([confirm], layout=form_item_layout),
    ]
    form2 = Box(form_items2, layout=Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='30%'
    ))
    out2 = widgets.Output()
    display(form2, out2)

    # Method that updates text field with README info if the dropdown value changes
    def on_change(change):
        path = names[change.new]
        if os.path.exists(os.path.join(path, 'README.md')):
            full_path = os.path.join(path, 'README.md')
            with open(full_path, 'r') as file:
                x = file.read()
        else:
            x = 'No README available'
        text.value = x
    
    # Method that runs when install button is clicked, tries to install demo
    def install(b):
        confirm.icon='spinner'
        confirm.description='Loading'
        confirm.button_style='warning'

        # Method that installs graph to server
        def load():
            os.chdir(names[dropdown.value])

            # Create install message to display
            loading = widgets.Label(
                        value=f'Demo installing. This can take up to 5 minutes',
                        disabled=False,
                        layout=Layout(width='50%')
                        )
            with out2:
                display(loading)

            # Create loading bar to display while loading demo
            with tqdm(total=100, leave=False) as pbar:
                pbar.bar_format="{percentage:3.0f}%|{bar:10}|{desc}" 
                confirm.icon='spinner'
                confirm.description='Loading'
                confirm.button_style='warning'

                # Look for bash file, if file exists run scripts found in file in order
                if glob('*.sh'):
                    commands = []
                    gsql_commands = []
                    with open(glob('*.sh')[0], 'r') as file:
                        for line in file.readlines():
                            if line.strip() != '':
                                    commands.append(line.strip())
                    for command in commands:
                        gsql_commands.append(command.split(' ')[1])
                    for command in gsql_commands:
                        with open(command, 'r') as file:
                            pbar.set_description_str('Running script: ' + command)      # Display script that is running
                            x = file.read()
                            script_lines = file.readlines()
                            if 'load' in command and 'run job' not in x and 'RUN JOB' not in x:     # Need to extra steps if script is loading data
                                
                                # Run script after adding additional lines to ensure it loads properly
                                case = re.search('create loading job', x, flags=re.IGNORECASE).end()
                                job_name = x[case:].strip().split(' ')[0]
                                token = graph.getToken(graph.createSecret())[0]
                                script_temp = []
                                for line in script_lines:
                                    if re.search('define filename', line, flags=re.IGNORECASE):
                                        index = line.find('=')
                                        v2 = line[:index]
                                        script_temp.append(v2.strip()+';\n')
                                    else:
                                        script_temp.append(line)
                                script = "".join(script_temp) + 'run job ' + job_name
                                graph.gsql(script, options=[])

                                # Run a load directly to ddl endpoint on server (This acutally loads the data in)
                                headers = {'Authorization': 'Bearer ' + token}
                                params = {'tag': job_name, 'filename': 'f'}
                                url = hostname.value + ":9000/ddl/" + graph.graphname
                                y = [m.start() for m in re.finditer('define filename', x, flags=re.IGNORECASE)]
                                data_list = {}
                                for index in y:
                                    new_string = x[index+16:]
                                    end = new_string.find('.csv') + 4
                                    string1 = new_string[:end]
                                    begin = string1.rfind('/')
                                    file_name = string1[begin+1:]
                                    data_list[new_string.strip().split(' ')[0]] = file_name
                                files = {}
                                for x in data_list.keys():
                                    params['filename'] = x
                                    files[x] = open(data_list[x], 'rb')
                                try:
                                    r = requests.post(url, headers=headers, files=files, params=params)
                                except Exception as e:
                                    out2.clear_output()
                                    with out2:
                                        display(error_load)
                                pbar.update(100/len(gsql_commands))
                            else:                                           # If script is not loading data, just run with no further edits
                                graph.gsql(file.read(), options=[])
                                pbar.update(100/len(gsql_commands))

                # If no bash file found, run scripts in order to best of ability
                else:
                    gsql_commands = []
                    temp = []
                    for file in glob('*.gsql'):
                        if 'schema' in file:
                            gsql_commands.insert(0, file)
                        elif 'load' in file:
                            gsql_commands.insert(1, file)
                        else:
                            temp.append(file)
                    for x in temp:
                        gsql_commands.append(x)
                    for command in gsql_commands:
                        with open(command, 'r') as file:
                            pbar.set_description_str('Running script: ' + command)      # Display script that is running
                            x = file.read()
                            script_lines = file.readlines()
                            if 'load' in command and 'run job' not in x and 'RUN JOB' not in x:         # Need to extra steps if script is loading data
                                
                                # Run script after adding additional lines to ensure it loads properly
                                case = re.search('create loading job', x, flags=re.IGNORECASE).end()
                                job_name = x[case:].strip().split(' ')[0]
                                token = graph.getToken(graph.createSecret())[0]
                                script_temp = []
                                for line in script_lines:
                                    if re.search('define filename', line, flags=re.IGNORECASE):
                                        index = line.find('=')
                                        v2 = line[:index]
                                        script_temp.append(v2.strip()+';\n')
                                    else:
                                        script_temp.append(line)
                                script = "".join(script_temp)+ 'run job ' + job_name
                                graph.gsql(script, options=[])

                                # Run a load directly to ddl endpoint on server (This acutally loads the data in)
                                headers = {'Authorization': 'Bearer ' + token}
                                params = {'tag': job_name}
                                url = hostname.value + ":9000/ddl/" + graph.graphname
                                y = [m.start() for m in re.finditer('define filename', x, flags=re.IGNORECASE)]
                                data_list = {}
                                for index in y:
                                    new_string = x[index+16:]
                                    end = new_string.find('.csv') + 4
                                    string1 = new_string[:end]
                                    begin = string1.rfind('/')
                                    file_name = string1[begin+1:]
                                    data_list[new_string.strip().split(' ')[0]] = file_name
                                files = {}
                                for x in data_list.keys():
                                    params['filename'] = x
                                    files[x] = open(data_list[x], 'rb')
                                try:
                                    r = requests.post(url, headers=headers, files=files, params=params)
                                except Exception as e:
                                    out2.clear_output()
                                    with out2:
                                        display(error_load)
                                pbar.update(100/len(gsql_commands))
                            else:                                               # If script is not loading data, just run with no further edits
                                graph.gsql(file.read(), options=[])     
                                pbar.update(100/len(gsql_commands))

                # Wrap up loading process, update buttons and loading bar
                pbar.set_description_str(' Connected')
                confirm.icon='check-circle'
                confirm.description='Success'
                confirm.button_style='success'
                time.sleep(1)
                pbar.close()
                confirm.disabled=True
                dropdown.disabled=True
                text.disabled=True
                out2.clear_output()

                # Create done message, display
                done = widgets.Label(
                    value=f'Demo successfully installed. You can check it out at {hostname.value}:14240',
                    disabled=False,
                    layout=Layout(width='50%')
                    )
                display(done)
        
        #--Confirm_Selection_BEGIN--#       Show buttons to continue or cancel if a graph is already on the server (since loading process will replace it)
        is_graph = None
        track=None
        graphname = None
        with io.capture_output() as captured:
            show = graph.gsql('ls', options=[])
        if 'Graphs:' in show:
            index = show.find('Graphs:')
            if '-Graph' in show[index:] or '- Graph' in show[index:]:
                is_graph = True
                temp = show[index:]
                index2 = temp.find('(')
                sup = temp[:index2].rfind(' ')
                graphname = temp[sup+1:index2]
                graph.graphname = graphname
        
        # Display button options of a graph already exists
        if is_graph is not None:     
            forming = [
                    Box([Label(value=f'A graph {graph.graphname} already exists. This Demo will replace it. Continue?')], layout=form_item_layout),
                    Box([graph_exists_cancel, graph_exists_continue], layout=form_item_layout),
                    ]
            former = Box(forming, layout=Layout(
                    display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='30%'
                ))
            with out2:
                display(former)
            
            # Create functions to wait until one of the buttons is clicked
            def yield_for_change(widget, attribute):
                def f(iterator):
                    @wraps(iterator)
                    def inner():
                        i = iterator()
                        def next_i(change):
                            try:
                                i.send(change)
                            except StopIteration as e:
                                widget.unobserve(next_i, attribute)
                        widget.on_click(next_i)
                        next(i)
                    return inner
                return f
            
            # If cancel is clicked, cancel upload
            @yield_for_change(graph_exists_cancel, 'value')
            def f():
                for i in range(1):
                    x=yield
                    confirm.icon='cloud-upload'
                    confirm.description='Install Demo'
                    confirm.button_style='info'
                    out2.clear_output()
                    track = False
            
            # If continue is clicked, run load() method
            @yield_for_change(graph_exists_continue, 'value')
            def g():
                for i in range(1):
                    x=yield
                    out2.clear_output()
                    load()
            f()
            g()
            if track is not None:
                return
        
        # If no graph is found (server is empty), then run load() method
        else:
            load()
            #--Confirm_Selection_END--#

    dropdown.observe(on_change, 'value')
    confirm.on_click(install)
    #--Display_Demo_Options_And_Load_END--#

# Create and display initial form (asks for hostname, username, etc.)
connect.on_click(connectToGraph)
form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='30%'
))
display(form, out)


Box(children=(Box(children=(Label(value='Hostname'), Text(value='', placeholder='TigerGraph Hostname (ex: http…

Output()